In [1]:
import sys
import pathlib
import pandas as pd
import numpy as np
import boto3

ROOT = pathlib.Path().absolute().parent.as_posix()
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [2]:
import pandas
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.session.Session().region_name

role = get_execution_role()

role = 'role_name_with_sagemaker_permissions' #i.e. my sagemaker role otherwise proceed inside sagemaker

sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0", role=role, instance_type="ml.t3.large", instance_count=1
)

Couldn't call 'get_role' to get Role ARN from role name joshg to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::991775080983:user/joshg, therefore it cannot be used as a SageMaker execution role

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code="unified_script.py",
    inputs=[ProcessingInput(source=input_data, destination="asdi-hackathon/*")],
    outputs=[ProcessingOutput(output_name="final_df", source="asdi-hackathon/final-data/final_data.csv")]
)

processing_job_description = sklearn_processor.jobs[-1].describe()
print(processing_job_description)